In [101]:
using Plots
using DataFrames
using Plots.PlotMeasures
using LaTeXStrings
using ColorSchemes
using LsqFit
using CSV


In [102]:
#df_C = DataFrame(times = times, C_5 = C_r_t_5,C_10 = C_r_t_10, C_20 = C_r_t_20, C_30 = C_r_t_30,
#C_40 = C_r_t_40,C_50 = C_r_t_50, SvN = SvN, chi=chi)


In [103]:
df = CSV.read("XYZ_heisenberg/C_array_MF.csv", DataFrame)
df_SvN = CSV.read("XYZ_heisenberg/SvN_array_MF.csv", DataFrame)
;
N = 201

201

In [104]:
SvN = real.(Matrix(df_SvN))
#SvN = Array{Float64}(SvN)

times = real.(df[!, "times"])
chi = df[!, "chi"]
C_r_t_5 = df[!, "C_5"]
C_r_t_10 = df[!, "C_10"]
C_r_t_20 = df[!, "C_20"]
C_r_t_30 = df[!, "C_30"]
C_r_t_40 = df[!, "C_40"]
C_r_t_50 = df[!, "C_50"]
C_r_t_60 = df[!, "C_60"]
C_r_t_70 = df[!, "C_70"]
C_r_t_80 = df[!, "C_80"]
C_r_t_90 = df[!, "C_90"]
;

In [105]:
gr()
heat = heatmap(1:(2*N), times, transpose(SvN), c = :seaborn_rocket_gradient,left_margin=40px,
    right_margin=40px, top_margin=40px,  bottom_margin = 40px, framestyle=:box, colorbar_title = "Entanglement Entropy")
ylabel!("Time t")
xlabel!("Site Index")

#title!("Entanglement Entropy system + ancilla")
savefig(heat, "heatmap.png")


# Plot the entanglement entropy for bonds separating system + ancilla pairs:
gr()
S = transpose(SvN)[:,2:2:(2*N)]
heat1 = heatmap(1:N, times, S, c = :seaborn_rocket_gradient,left_margin=40px,
    right_margin=40px, top_margin=40px,  bottom_margin = 40px, framestyle=:box, colorbar_title = "Entanglement Entropy")
ylabel!("Time t")
xlabel!("Site Index")
#title!("Entanglement Entropy Separating System and Ancilla")
savefig(heat1,"heatmap_bonds_sep.png")

# Plot entanglement entropy of bonds between system + ancilla pairs:
gr()
S = transpose(SvN)[:,1:2:(2*N)]
heat2 = heatmap(1:N, times, S,c = :seaborn_rocket_gradient,left_margin=40px,
    right_margin=40px, top_margin=40px,  bottom_margin = 40px, framestyle=:box, colorbar_title = "Entanglement Entropy")
ylabel!("Time t")
xlabel!("Site Index")
#title!("Entanglement Entropy Between System and Ancilla")
savefig(heat2,"heatmap_bonds_between.png")

"c:\\Users\\rache\\quantum_scrambling\\src\\heatmap_bonds_between.png"

In [106]:
# Plot the growth in the maximum link dimension with time:
plot(times, chi, label=false,left_margin=20px, right_margin=20px, top_margin=20px,  bottom_margin = 20px, framestyle=:box, linecolor=get(ColorSchemes.seaborn_rocket_gradient,0.1))  
xlabel!("Time")
ylabel!(L"$\chi$")
#scatter = scatter!(times, chi, label=false) 
savefig("scatter.png")


"c:\\Users\\rache\\quantum_scrambling\\src\\scatter.png"

In [107]:

plot(times,log.(C_r_t_5), label="r=5", left_margin=20px,top_margin = 20px,
    right_margin=20px, bottom_margin = 20px, framestyle=:box, linecolor=get(ColorSchemes.seaborn_rocket_gradient,0.2))#, ylim=(min_y, 20)) #, ylim=(min_y, 20)

plot!(times,log.(C_r_t_10), label="r=10", linecolor=get(ColorSchemes.seaborn_rocket_gradient,0.3))#, ylim=(min_y, 20)) #!

plot!(times,log.(C_r_t_20), label="r=20", linecolor=get(ColorSchemes.seaborn_rocket_gradient, 0.4))#, ylim=(min_y, 20))

plot!(times,log.(C_r_t_30), label="r=30", linecolor=get(ColorSchemes.seaborn_rocket_gradient, 0.5))#, ylim=(min_y, 20))

plot!(times,log.(C_r_t_40), label="r=40", linecolor=get(ColorSchemes.seaborn_rocket_gradient,0.6))#, ylim=(min_y, 20))

plot!(times,log.(C_r_t_50), label="r=50", linecolor=get(ColorSchemes.seaborn_rocket_gradient, 0.65))#, ylim=(min_y, 20))

xlabel!("Time")
ylabel!("log(C(r,t))")

savefig("crt.png")


"c:\\Users\\rache\\quantum_scrambling\\src\\crt.png"

In [158]:
filt = filter( x -> x > 25.0 && x < 25.05, times ) 
start_index = indexin( filt[1], times)
start_index = start_index[1]

filt = filter( x -> x > 125.0 && x < 125.05, times ) 
stop_index = indexin( filt[1], times)
stop_index = stop_index[1]

2501

In [168]:
R = 70
# Define the function to fit
function C_early(t, λ, p, v_B; r=R)
    bracket = @.(-λ*(((r/v_B)-t)^(1 +p))/(t^p))
    return exp.(bracket)
end


# Define the residual function for least squares fitting
function residual(params, t, C_data)
    λ, p, v_B = params
    model_C = C_early(t, λ, p, v_B; r=R)  
    return vec(C_data .- model_C)
end




residual (generic function with 1 method)

In [169]:
# Initial guess for the parameters
p0 = [1.0, 1.0, 1.0]  # initial guess for λ, p, v_B

time_data = times[start_index:stop_index]
C_data = C_r_t_70[start_index:stop_index]
# Perform the least squares fit
fit_result = curve_fit(residual, time_data, C_data, p0)

# Extract the fitted parameters
λ_fit, p_fit, v_B_fit = fit_result.param

MethodError: MethodError: no method matching residual(::Vector{Float64}, ::Vector{Float64})

Closest candidates are:
  residual(::Any, ::Any, !Matched::Any)
   @ Main c:\Users\rache\quantum_scrambling\src\read_csv.ipynb:10


In [160]:
# fit c(r,t) to c_early

function C_early(r,t, λ, p, v_B)
    
    bracket = -λ*(((r/v_B)-t)^(1+p))/(t^p)
    return exp(bracket)
end

# input = t
# params = [λ,p,v_B]
r = 70
c_early(t, params) = exp.(Complex.(-params[1]*(((r/params[3]).-t))).^(1 .+ params[2])/(Complex.(t).^params[2]))

p0 = [1.9,0.67,0.67]
fit = curve_fit(c_early, times[start_index:stop_index], C_r_t_70[start_index:stop_index], p0)
p_new = fit.param

DimensionMismatch: DimensionMismatch: dimensions must match: a has dims (Base.OneTo(2002), Base.OneTo(2002)), must have singleton at dim 2